In [10]:
def format_product(properties):
    if 'Размер упаковки' in properties:
        if properties["Размер упаковки"] is not None:
            dimensions = properties['Размер упаковки'].replace(" см", "").split(" x ")
            properties.pop("Размер упаковки")
            properties["Длина упаковки"] = dimensions[0]
            properties["Ширина упаковки"] = dimensions[1]
            properties["Высота упаковки"] = dimensions[2]
        else:
            properties.pop("Размер упаковки")
            properties["Длина упаковки"] = 'Not provided'
            properties["Ширина упаковки"] = 'Not provided'
            properties["Высота упаковки"] = 'Not provided'
            
    if "Срок годности" in properties:
        if properties["Срок годности"] is not None:
            l = len(properties["Срок годности"])
            properties["Срок годности"] = properties["Срок годности"].split(' ')[2]
        else:
            properties["Срок годности"] = 'Not provided'

    if "Вес упаковки" in properties:
        if properties["Вес упаковки"] is not None:
            properties["Вес упаковки"] = properties["Вес упаковки"].split(' ')[0]
        else:
            properties["Вес упаковки"] = 'Not provided' 

    if "Количество в упаковке" in properties:
         if properties["Количество в упаковке"] is not None:
            value = properties["Количество в упаковке"]
            properties["Количество в упаковке"] = value.split(' ')[0]
            properties["Единица измерения"] = value.split(' ')[1]   
         else:
             properties["Количество в упаковке"] = 'Not provided'
             properties["Единица измерения"] = 'Not provided'
             
    return properties

In [18]:
product_properties_searching = ['Код продукта', 'Название продукта', 'Бренд', 'Срок годности', 'Количество в упаковке', 'Единица измерения', 'Вес упаковки', 'Цена', 'Размер упаковки', 'Категория']
product_properties_resulting = ['Код продукта', 'Название продукта', 'Бренд', 'Срок годности', 'Количество в упаковке', 'Единица измерения', 'Вес упаковки', 'Цена', 'Категория', 'Длина упаковки', 'Ширина упаковки', 'Высота упаковки']

def from_html(driver, link):
    driver.get(link)
    
    try:
        properties_section = driver.find_element(By.CSS_SELECTOR, "div.product__properties-items")
        
        property_blocks = properties_section.find_elements(By.CLASS_NAME, "product__property")
        
        properties = dict.fromkeys(product_properties_searching)
        
        for block in property_blocks:
            property_name = block.find_element(By.CLASS_NAME, "product__property-name").text.strip()
            property_name = property_name[:len(property_name)-1]
            if property_name in product_properties_searching:
                property_value = block.find_element(By.CLASS_NAME, "product__property-value").text.strip()
                properties[property_name] = property_value
            else: 
                print(f"Не нашли {property_name} в {properties}")
            
    except Exception as e:
        print("Не удалось найти свойства товара", e)

    try:
        price = driver.find_element(By.XPATH, "/html/body/div[2]/main/div[2]/div/form/div[9]/div/div[1]/span[2]").text
        properties["Цена"] = price
    except Exception as e:
        print("Не удалось найти цену товара", e)

    try:
        product_name = driver.find_element(By.XPATH, "/html/body/div[2]/main/div[2]/div/form/div[2]/h1").text
        properties["Название продукта"] = product_name
    except Exception as e:
        print("Не удалось найти название товара", e)

    return properties

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()

product = format_product(from_html(driver, "https://iherb-russia.org/product/protocol-for-life-balance-adrenal-cortisol-support-s-relora-preparat-dlya-podderzhki-nadpochechnikov-90-rastitelnyh-kapsul"))

print(product)

{'Код продукта': 'PRT-13344', 'Название продукта': 'Protocol for Life Balance, Adrenal Cortisol Support с Relora, препарат для поддержки надпочечников, 90 растительных капсул', 'Бренд': 'Protocol for Life Balance', 'Срок годности': 'Not provided', 'Количество в упаковке': '90', 'Единица измерения': 'шт.', 'Вес упаковки': '0.09', 'Цена': '5 424 ₽', 'Категория': None, 'Длина упаковки': '10.5', 'Ширина упаковки': '5.1', 'Высота упаковки': '4.8'}


In [19]:
def get_categories_of_supplements(driver):
    categories = {}
    url = 'https://iherb-russia.org'
    driver.get(url)
    
    driver.implicitly_wait(2)
    
    try:
        for i in range(1, len(driver.find_elements(By.XPATH, "/html/body/div[2]/header/div[1]/div/div[1]/div/div[4]/ul/li[1]/ul/div/li")) + 1):
            item = driver.find_element(By.XPATH, "/html/body/div[2]/header/div[1]/div/div[1]/div/div[4]/ul/li[1]/ul/div/li[" + str(i) + ']/div/a')
            
            link = item.get_attribute('href')
            space = item.get_attribute('innerHTML').strip().find('  ')
            if space != -1:
                name = item.get_attribute('innerHTML').strip()[:space].rstrip()
            else:
                name = item.get_attribute('innerHTML').strip()
            categories[name] = link
        return categories
    except Exception as e:
        print("Не удалось найти категории", e)

In [20]:
def parse_category(driver, link, category_name):
    all_products = []

    driver.get(link)
    try:
        number_of_pages = driver.find_element(By.XPATH, "/html/body/div[2]/main/div[3]/div/div/div/a[3]")
    except Exception as e:
        try:
            number_of_pages = driver.find_element(By.XPATH, "/html/body/div[2]/main/div[3]/div/div/div/a[2]")
        except Exception as e:
            try:
                number_of_pages = driver.find_element(By.XPATH, "/html/body/div[2]/main/div[3]/div/div/div/a[1]")
            except Exception as e:
                number_of_pages = 1
                
    for page_number in range (1, int(number_of_pages.text) + 1):
        print(f"Приступаем к парсингу страницы №{page_number}")
        
        driver.get(link + "?page=" + str(page_number))
        number_of_products_on_the_page = len(driver.find_elements(By.CLASS_NAME, 'product-preview-elem'))
        
        products_links_we = [driver.find_element(By.XPATH, "/html/body/div[2]/main/div[2]/div/div[2]/div/div[" + str(product_number) + "]/form/div/div[2]/div/a") for product_number in range(1, number_of_products_on_the_page+1) ]
        product_links = [i.get_attribute('href') for i in products_links_we]
        print(f"product_links={product_links}")
        for i in range(1, len(product_links) + 1):
            print(f"Парсим элемент №{i}")
            product = format_product(from_html(driver, product_links[i-1]))
            product["Категория"] = category_name
            all_products.append(product)
            
        
    return all_products

In [21]:
import pandas as pd

driver = webdriver.Chrome()

categories = get_categories_of_supplements(driver)

data = []

print(product_properties_resulting)
wrong_products = []
for name, link in categories.items():
    print(f"Приступаем к парсингу категории {name}")
    
    category_products = parse_category(driver, link, name)
    for product in category_products:
        
        if list(product.keys()) == product_properties_resulting:
            data.append(list(product.values()))
        else:
            print("^ Элемент не подходит ^")
            wrong_products.append(product)
    break
driver.quit()

data_frame = pd.DataFrame(data, columns = product_properties_resulting)

data_frame.to_csv("untitled(2).csv")

['Код продукта', 'Название продукта', 'Бренд', 'Срок годности', 'Количество в упаковке', 'Единица измерения', 'Вес упаковки', 'Цена', 'Категория', 'Длина упаковки', 'Ширина упаковки', 'Высота упаковки']
Приступаем к парсингу категории Аминокислоты
Приступаем к парсингу страницы №1
product_links=['https://iherb-russia.org/product/primaforce-poroshok-d-asparaginovoy-kisloty-bez-dobavok-300-g-106-untsii', 'https://iherb-russia.org/product/nutricost-d-asparaginovaya-kislota-bez-vkusovyh-dobavok-300-g-106-untsii', 'https://iherb-russia.org/product/bluebonnet-nutrition-nucleotide-complex-rnk-dnk-60-rastitelnyh-kapsul', 'https://iherb-russia.org/product/swanson-l-gistidin-500-mg-60-rastitelnyh-kapsul', 'https://iherb-russia.org/product/kirkman-labs-dmg-liquid-naturalnaya-malina-473-ml-16-zhidk-untsiy', 'https://iherb-russia.org/product/allergy-research-group-trimetilglitsin-tmg-100-rastitelnyh-kapsul', 'https://iherb-russia.org/product/solgar-dlpa-v-svobodnoy-forme-500-mg-100-rastitelnyh-kaps

IndexError: list index out of range

In [23]:
print(data)

[]
